## 종목별 주가 데이터 크롤링

In [ ]:
import requests    # browser 와 server 간 통신
import pandas as pd    # 데이터 분석 도구

In [ ]:
# 데이터 포맷팅
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('mode.chained_assignment', None)

In [ ]:
# url: 서버 주소
url = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader',
    'Origin': 'http://data.krx.co.kr',
}

## 종목 마스터 크롤링

In [ ]:
# 종목마스터
data = {
    'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
    'locale': 'ko_KR',
    'mktId': 'ALL',
    'share': '1',
    'csvxls_isNo': 'false',
}
raw = requests.post(url, data=data, headers=headers)    # 서버와 통신
rst = raw.json()['OutBlock_1']    # 딕셔너리로 변환 후 'output' 의 value만 추출
ln = []
for r in rst:
    ln.append([c for c in r.values()])
df_master = pd.DataFrame(ln)
df_master.columns = r.keys()
df_master.tail()

In [ ]:
df_master['ISU_ABBRV']=='삼성전자'

In [ ]:
df_master[df_master['ISU_ABBRV']=='삼성전자']

In [ ]:
df_master[df_master['ISU_ABBRV']=='삼성전자']['ISU_CD'].iloc[0]

In [ ]:
df_master[df_master['ISU_ABBRV']=='삼성전자']['ISU_CD'].iloc[0]

In [ ]:
df_master[df_master['ISU_SRT_CD']=='005930']['ISU_ABBRV'].iloc[0]

## 모든 종목에 쓸 수 있도록 함수화

In [ ]:
def historical_price(symbol, start_date, end_date):
    # 종목정보 선택
    stock = df_master[df_master['ISU_SRT_CD']==symbol] if len(df_master[df_master['ISU_ABBRV']==symbol.upper()])<1 else df_master[df_master['ISU_ABBRV']==symbol.upper()]
    print(stock[['ISU_SRT_CD', 'ISU_ABBRV']])
    if len(stock)>0:
        # 입력인자 세팅
        payload = {
            'bld': 'dbms/MDC/STAT/standard/MDCSTAT01701',
            'isuCd': '{}'.format(stock['ISU_CD'].iloc[0]),
            'strtDd': start_date,
            'endDd': end_date,
        }
        raw = requests.post(url, data=payload, headers=headers)    # 서버와 통신
        rst = raw.json()['output']    # 딕셔너리로 변환 후 'output' 의 value만 추출
        df = pd.DataFrame.from_dict(rst)    # 딕셔너리를 데이터프레임으로 변환
        return df
    else:
        return 'No matched result'

In [ ]:
df = historical_price(symbol='005930', start_date='20241101', end_date='20241130')
df

In [ ]:
df.rename(columns={'TRD_DD':'Date', 'TDD_OPNPRC':'Open', 'TDD_HGPRC':'High', 'TDD_LWPRC':'Low', 'TDD_CLSPRC':'Close', 'ACC_TRDVOL':'Volume', 'ACC_TRDVAL':'Value', 'MKTCAP':'MarketCap', 'LIST_SHRS':'Shares',}, inplace=True)
df

In [ ]:
df.drop(columns=['FLUC_TP_CD', 'CMPPREVDD_PRC', 'FLUC_RT'], inplace=True)
df

In [ ]:
df['Date']

In [ ]:
pd.to_datetime(df['Date'])

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date']

In [ ]:
df['Close']

In [ ]:
df['Close'][0] + df['Close'][1]

In [ ]:
df['Close'].str.replace(',','')

In [ ]:
df['Close'].str.replace(',','').astype(float)

In [ ]:
df['Close'] = df['Close'].str.replace(',','').astype(float)

In [ ]:
df['Close']

In [ ]:
df['Close'][0] + df['Close'][1]

In [ ]:
df

In [ ]:
df.set_index('Date', inplace=True)
df

In [ ]:
df.loc['2024-11-05']